## Importing the dataset

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import joblib

# pd.set_option('display.max_rows', None)

In [20]:
df = pd.read_csv('dataset/recipes.csv')

In [21]:
df

,id,name,instructions,ingredients,description,tags
0,106,Mushroom Risotto,Molestiae dolores quod voluptate aut rerum maiores repellat. Nihil aut eum dolores. Dolor modi nemo cumque qui nesciunt qui vel. Temporibus quidem rem corrupti. Adipisci cupiditate quis voluptatem molestiae.,Pêra Manjerona Anchovas Coentro,Aperiam saepe odio fugit delectus occaecati et.,TAG43 TAG49 TAG47 TAG7 TAG91
1,107,Chicken Parm,Debitis dolore vitae. Provident consequatur nostrum possimus. Repellendus provident quasi architecto. Velit harum illum voluptas.,Jewfish Semente Abóbora Vieiras Semente Ajowan,Est vel ut aut vel repellat qui.,TAG3 TAG7 TAG96 TAG58 TAG68
2,108,Arepas,Nihil omnis consequuntur qui eveniet. Culpa quaerat iste qui rem molestias. Ab aliquid nesciunt nam itaque perspiciatis.,Manjerona Pão Montanha Salsisha Maçã,Ea rerum quis quod est ratione sit qui.,TAG27 TAG42 TAG13 TAG58 TAG83
3,109,Pho,Et dolor cum. Consequatur soluta vero dolor iste est. Magni deserunt et ut quo omnis. Distinctio unde quo quae.,Pêra Fermento Em Pó Mandarins Lentilha Vermelha,Maxime inventore quis consequatur maxime velit pariatur.,TAG31 TAG73 TAG1 TAG67 TAG5
4,110,Caprese Salad,Perferendis illo eveniet ab voluptas et et atque. Et repellat modi quae cumque. Eligendi odit et.,Bacon Farinha Semente Ajowan Vinagre Vinho Tinto,Ut illum repellendus unde sapiente fugit est repellendus.,TAG94 TAG91 TAG19 TAG4 TAG8
...,...,...,...,...,...,...
995,1101,Tiramisù,Tenetur nihil sunt dignissimos ex quia. Exercitationem ipsam dolor in. Totam deserunt quia eos.,Soro Leite Farinha Tapioca Canela Alface,Vitae aut qui doloremque ut maiores error ea.,TAG57 TAG99 TAG6 TAG74 TAG60
996,1102,Stinky Tofu,Mollitia delectus eligendi adipisci atque ea. Et quia quia assumenda est. Sed omnis non sit velit quia reiciendis.,Farinha Grão Bico Pêssego Berinjela Farinha Trigo,Vitae quo optio.,TAG40 TAG68 TAG0 TAG0 TAG42
997,1103,Caprese Salad,Et cum aliquam illum aliquid. Eos perspiciatis nihil a. Maiores perferendis ducimus. Harum voluptatum corporis omnis ab vero et ratione.,Uva Queijo Cabra Damascos Molho Ostras,Accusamus ducimus itaque perferendis culpa ullam eveniet.,TAG8 TAG25 TAG96 TAG15 TAG47
998,1104,Tacos,Nostrum vero recusandae distinctio suscipit earum voluptatem. Voluptatibus necessitatibus velit aspernatur. Suscipit id voluptas excepturi aut sit tempore.,Semente Ajowan Sardinha Beringela Quinoa,Ut ipsam eum totam maiores excepturi suscipit corrupti.,TAG44 TAG7 TAG4 TAG26 TAG84


In [22]:
content_columns = ['tags', 'ingredients']
all_columns = ['name'] + content_columns
df2 = df[all_columns]
df2.head()

,name,tags,ingredients
0,Mushroom Risotto,TAG43 TAG49 TAG47 TAG7 TAG91,Pêra Manjerona Anchovas Coentro
1,Chicken Parm,TAG3 TAG7 TAG96 TAG58 TAG68,Jewfish Semente Abóbora Vieiras Semente Ajowan
2,Arepas,TAG27 TAG42 TAG13 TAG58 TAG83,Manjerona Pão Montanha Salsisha Maçã
3,Pho,TAG31 TAG73 TAG1 TAG67 TAG5,Pêra Fermento Em Pó Mandarins Lentilha Vermelha
4,Caprese Salad,TAG94 TAG91 TAG19 TAG4 TAG8,Bacon Farinha Semente Ajowan Vinagre Vinho Tinto


In [23]:
df2['Attributes'] = df[content_columns].astype(str).agg(' '.join, axis=1)

C:\Users\rafae\AppData\Local\Temp\ipykernel_23464\3834045754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Attributes'] = df[content_columns].astype(str).agg(' '.join, axis=1)


In [24]:
pd.set_option('display.max_colwidth', None)
df2['Attributes'].head()

0                    TAG43 TAG49 TAG47 TAG7 TAG91 Pêra Manjerona Anchovas Coentro
1      TAG3 TAG7 TAG96 TAG58 TAG68 Jewfish Semente Abóbora Vieiras Semente Ajowan
2              TAG27 TAG42 TAG13 TAG58 TAG83 Manjerona Pão Montanha Salsisha Maçã
3     TAG31 TAG73 TAG1 TAG67 TAG5 Pêra Fermento Em Pó Mandarins Lentilha Vermelha
4    TAG94 TAG91 TAG19 TAG4 TAG8 Bacon Farinha Semente Ajowan Vinagre Vinho Tinto
Name: Attributes, dtype: object

In [25]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df2['Attributes'])
tfidf_matrix.shape

(1000, 358)

In [26]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
def get_recommendations(RecipeName):
    idx = df2.index[df2['name'] == RecipeName].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    recipe_indices = [i[0] for i in sim_scores]
    return df2[['name', 'tags', 'ingredients']].iloc[recipe_indices]

In [37]:
name = df2.sample().iloc[0]['name']
display(df2.loc[df2['name'] == name])

,name,tags,ingredients,Attributes
84,Poutine,TAG84 TAG45 TAG76 TAG84 TAG89,Noodles Asiáticos Fígado Brotos Queijo Parmesão,TAG84 TAG45 TAG76 TAG84 TAG89 Noodles Asiáticos Fígado Brotos Queijo Parmesão
110,Poutine,TAG26 TAG67 TAG94 TAG91 TAG19,Arroz Rins Quinoa Mandarins,TAG26 TAG67 TAG94 TAG91 TAG19 Arroz Rins Quinoa Mandarins
277,Poutine,TAG89 TAG15 TAG82 TAG9 TAG19,Orégano Canela Lentilha Sardinha,TAG89 TAG15 TAG82 TAG9 TAG19 Orégano Canela Lentilha Sardinha
289,Poutine,TAG45 TAG20 TAG79 TAG91 TAG62,Lichia Couve Cenoura Roxa Framboesa,TAG45 TAG20 TAG79 TAG91 TAG62 Lichia Couve Cenoura Roxa Framboesa
310,Poutine,TAG47 TAG95 TAG90 TAG24 TAG49,Manga Nectarinas Caqui Achacha,TAG47 TAG95 TAG90 TAG24 TAG49 Manga Nectarinas Caqui Achacha
363,Poutine,TAG49 TAG98 TAG33 TAG96 TAG63,Hortelã Bérberis Goiaba Alho Poró,TAG49 TAG98 TAG33 TAG96 TAG63 Hortelã Bérberis Goiaba Alho Poró
472,Poutine,TAG98 TAG37 TAG19 TAG24 TAG34,Pimenta Chili Broto Feijão Óleo Milho Orégano,TAG98 TAG37 TAG19 TAG24 TAG34 Pimenta Chili Broto Feijão Óleo Milho Orégano
512,Poutine,TAG19 TAG38 TAG58 TAG20 TAG4,Leite Arroz Semente Papoila Cebolinha Sábio,TAG19 TAG38 TAG58 TAG20 TAG4 Leite Arroz Semente Papoila Cebolinha Sábio
551,Poutine,TAG66 TAG2 TAG38 TAG56 TAG23,Alcaparras Sardinha Polvo Queijo Prato,TAG66 TAG2 TAG38 TAG56 TAG23 Alcaparras Sardinha Polvo Queijo Prato
569,Poutine,TAG91 TAG68 TAG70 TAG92 TAG33,Coentro Cenoura Cheddar Rins,TAG91 TAG68 TAG70 TAG92 TAG33 Coentro Cenoura Cheddar Rins


In [28]:
index = df2.index[df2['name'] == name].tolist()[0]
display(df2[df2.index == index])

,name,tags,ingredients,Attributes
7,Pizza,TAG2 TAG99 TAG80 TAG87 TAG43,Creme Leite Ovo Alho Alcaçuz,TAG2 TAG99 TAG80 TAG87 TAG43 Creme Leite Ovo Alho Alcaçuz


In [30]:
get_recommendations(name)

,name,tags,ingredients
225,California Maki,TAG84 TAG2 TAG53 TAG43 TAG87,Alho Poró Berinjela Linguado Brócolis
964,Hummus,TAG38 TAG4 TAG83 TAG69 TAG2,Barramundi Ricotta Ovo Alcaçuz
768,Philadelphia Maki,TAG49 TAG30 TAG99 TAG25 TAG23,Ovo Alho Vinagre Queijo Cabra
198,Pappardelle alla Bolognese,TAG33 TAG87 TAG91 TAG52 TAG89,Barramundi Tortilla Milho Leite Soja Creme Leite
991,Pappardelle alla Bolognese,TAG63 TAG27 TAG50 TAG5 TAG51,Macarrão Alho Pato Creme Leite
543,Pho,TAG44 TAG61 TAG62 TAG90 TAG83,Vieiras Alho Geléia Creme Leite
875,Lasagne,TAG80 TAG2 TAG8 TAG22 TAG43,Feijão Branco Lentilha Repolho Cebolinha
355,Pork Belly Buns,TAG32 TAG17 TAG2 TAG22 TAG7,Lentilha Creme Leite Kiwi Chagas
279,Pasta Carbonara,TAG87 TAG99 TAG58 TAG87 TAG28,Fubá Amora Brotos Amaranto
487,Ebiten maki,TAG88 TAG1 TAG93 TAG88 TAG80,Batata Feijão Creme Leite Uva


In [12]:
def get_recommendations_by_list_names(recipe_names):

    combined_ranking = {}
    sorted_combined_ranking = {}
    count = 10
    recipe_indices = []

    for recipe_name in recipe_names:
        idx = df2.index[df2["name"] == recipe_name].tolist()[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]

        for i, recipe in enumerate(sim_scores):
            combined_ranking[recipe] = combined_ranking.get(recipe, 0) + i

        sorted_combined_ranking = sorted(combined_ranking.items(), key=lambda x: x[1])

    top10sorted_combined_ranking = sorted_combined_ranking[-count:]

    for i in top10sorted_combined_ranking:
        recipe_indices.append(i[0][0])
    
    # display(sorted_combined_ranking)
    print(df2[['name', 'tags', 'ingredients']].iloc[recipe_indices])
    print('\n')

In [46]:
indexes = df.sample(5).index.tolist()
df2.iloc[indexes]
    

,name,tags,ingredients,Attributes
516,Poke,TAG78 TAG95 TAG18 TAG80 TAG68,Cogumelo Sementes Gergelim Brotos Pimenta,TAG78 TAG95 TAG18 TAG80 TAG68 Cogumelo Sementes Gergelim Brotos Pimenta
923,Pizza,TAG93 TAG34 TAG54 TAG63 TAG71,Amêndoas Edamame Raspas Limão Rabanete,TAG93 TAG34 TAG54 TAG63 TAG71 Amêndoas Edamame Raspas Limão Rabanete
810,Souvlaki,TAG83 TAG65 TAG54 TAG85 TAG30,Óleo Amêndoa Chá Verde Lemongrass Coentro,TAG83 TAG65 TAG54 TAG85 TAG30 Óleo Amêndoa Chá Verde Lemongrass Coentro
287,Cauliflower Penne,TAG35 TAG22 TAG99 TAG23 TAG34,Vinagre Arroz Integral Grão Bico Pimenta Jamaica Mostarda,TAG35 TAG22 TAG99 TAG23 TAG34 Vinagre Arroz Integral Grão Bico Pimenta Jamaica Mostarda
735,Chicken Parm,TAG78 TAG19 TAG74 TAG88 TAG28,Frango Alcaparras Flores Lavanda Pato,TAG78 TAG19 TAG74 TAG88 TAG28 Frango Alcaparras Flores Lavanda Pato


In [49]:
names = df2.iloc[indexes]['name'].tolist()

In [50]:
get_recommendations_by_list_names(names)

                     name                           tags  \
249     Cauliflower Penne   TAG47 TAG90 TAG72 TAG4 TAG38   
279       Pasta Carbonara  TAG87 TAG99 TAG58 TAG87 TAG28   
249     Cauliflower Penne   TAG47 TAG90 TAG72 TAG4 TAG38   
826  Meatballs with Sauce  TAG96 TAG62 TAG72 TAG93 TAG95   
957               Lasagne   TAG67 TAG65 TAG26 TAG41 TAG3   
909        Vegetable Soup    TAG9 TAG71 TAG78 TAG9 TAG28   
487           Ebiten maki   TAG88 TAG1 TAG93 TAG88 TAG80   
226           Stinky Tofu  TAG77 TAG13 TAG66 TAG10 TAG54   
596           Katsu Curry   TAG1 TAG96 TAG97 TAG16 TAG42   
464       Chicken Fajitas  TAG93 TAG25 TAG58 TAG32 TAG60   

                                           ingredients  
249          Molho Ostras Cevada Lichia Feijão Carioca  
279                         Fubá Amora Brotos Amaranto  
249          Molho Ostras Cevada Lichia Feijão Carioca  
826  Noodles Asiáticos Farinha Rosca Molho Peixe Caqui  
957       Cacau Em Pó Achacha Semente Abóbora Sardinha